In [1]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import pandas as pd

1. Командная строка: в поиске  cmd
2. Посмотреть содержимое: dir
3. Комадна для смены дерриктории: cd /название_папки
4. Для того чтобы подняться вверх по каталогу: cd ..
5. Для включения питона набрать: jupyter notebook

In [2]:
a = np.array([[1,2,3],
              [1,1,1],
              [0,0,1]])

In [3]:
print("Матрица А:\n",a)
print("Матрица A транспонированная:\n",a.T)
print("Матрица обратная A:\n", np.linalg.inv(a))
print("Матрица A^2:\n", np.dot(a,a))

Матрица А:
 [[1 2 3]
 [1 1 1]
 [0 0 1]]
Матрица A транспонированная:
 [[1 1 0]
 [2 1 0]
 [3 1 1]]
Матрица обратная A:
 [[-1.  2.  1.]
 [ 1. -1. -2.]
 [ 0.  0.  1.]]
Матрица A^2:
 [[3 4 8]
 [2 3 5]
 [0 0 1]]


In [4]:
def distance_between_points(lat_a, lon_a, lat_b, lon_b):

    R = 6373.0

    lat1 = radians(lat_a)
    lon1 = radians(lon_a)
    lat2 = radians(lat_b)
    lon2 = radians(lon_b)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [5]:
df = pd.read_excel('Координаты.xlsx')

In [6]:
df.head()

,RegionNameORIGIN,Origin Latitude,Origin Longitude
0,Республика Башкортостан,54.734856,55.957856
1,Республика Бурятия,51.833507,107.584125
2,Республика Дагестан,42.983060,47.504682
3,Кабардино-Балкарская республика,43.484605,43.607090
4,Республика Калмыкия,46.308309,44.270181


In [7]:
#
lat_lon = df[['Origin Latitude','Origin Longitude']].values

In [8]:
matr_dist = np.zeros((79,79,79))

# Выбор h
1. $AIC = 2k+n[ln(2\pi RSS/n) +1]$
2. $RSS = \sum_{i=1}^{N}(y_i-h)^2$

In [9]:
for i in range(79):
    for j in range(79):
        matr_dist[i,j,j] = distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                                   lat_lon[j,0],lat_lon[j,1])

In [10]:
h = 0
for i in range(79):
    for j in range(79):
        h += (distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                 lat_lon[j,0],lat_lon[j,1])-2419.54)**2/79/79

In [11]:
h = h**(0.5)

In [12]:
h

1968.5992536211638

In [13]:
W = np.zeros((79,79,79))
for i in range(79):
    for j in range(79):
        W[i,j,j] = np.exp((-0.5)*(distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                                   lat_lon[j,0],lat_lon[j,1])/h)**2)

In [14]:
x_df = pd.read_excel('данные 2002 и 2016 год.xlsx')

In [15]:
x_df_2002 = x_df[['TFR','Gender gap', 'Secularisation', 'GDP per inhabitant']]
x_df_2016 = x_df[['Unnamed: 1','Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7']]

In [16]:
x_df_2002['GDP sqare'] = x_df_2002['GDP per inhabitant']*x_df_2002['GDP per inhabitant']

/home/artyom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
x_df_2002 = x_df_2002[1:]

In [18]:
x_df_2016['GDP sqare'] = x_df_2016['Unnamed: 7']*x_df_2016['Unnamed: 7']

/home/artyom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
x_df_2016 = x_df_2016[1:]

In [20]:
x_df_2002 = x_df_2002 - x_df_2002.mean()
x_df_2016 = x_df_2016 - x_df_2016.mean()

In [21]:
x_df_2002 = x_df_2002/x_df_2002.std()
x_df_2016 = x_df_2016/x_df_2016.std()

In [22]:
x_df_2002['const'] = 1
x_df_2016['const'] = 1

In [47]:
X_2002 = x_df_2002[['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare']].values
X_2016 = x_df_2016[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7','GDP sqare']].values

In [48]:
Y_2002 = x_df_2002.TFR.values
Y_2016 = x_df_2016['Unnamed: 1'].values

In [58]:
import numpy as np
import pandas as pd
import geopandas as gp
import pysal as ps
import matplotlib
import matplotlib.pyplot as plt
%pylab inline
from pysal.contrib.gwr.sel_bw import Sel_BW
from pysal.contrib.gwr.gwr import GWR
from pysal.contrib.glm.family import Gaussian


Populating the interactive namespace from numpy and matplotlib


In [50]:
coords = []
for i in lat_lon:
    coords.append((i[0],i[1]))

In [96]:
coords
bw_2002 = Sel_BW(lat_lon, Y_2002.reshape(len(Y_2002),1), X_2002, kernel='bisquare', fixed=False)
bws_2002 = bw_2002.search(search='golden_section', criterion='AICc')
#XB_2002 = bw_2002.XB
#err_2002 = bw_2002.err
print(bws_2002)
results = GWR(lat_lon, Y_2002.reshape(len(Y_2002),1), X_2002,bws_2002, constant=True).fit()

53.0


In [97]:
out_df_2002 = pd.DataFrame()
for i in results.params:
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare'],i))
    out_df_2002 =  out_df_2002.append(_answer,ignore_index=True)
    
out_df_2002 = out_df_2002.set_index(x_df_2002.index)
out_df_2002.to_excel('coef_2002.xlsx')

tval_df_2002 = pd.DataFrame()
for i in results.tvalues:
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare'],i))
    tval_df_2002 =  tval_df_2002.append(_answer,ignore_index=True)
    
tval_df_2002 = tval_df_2002.set_index(x_df_2002.index)
tval_df_2002.to_excel('tval_2002.xlsx')

In [98]:
coords
bw_2016 = Sel_BW(lat_lon, Y_2016.reshape(len(Y_2016),1), X_2016, kernel='bisquare', fixed=False)
bws_2016 = bw_2016.search(search='golden_section', criterion='AICc')
#XB_2002 = bw_2002.XB
#err_2002 = bw_2002.err
print(bws_2016)
results = GWR(lat_lon, Y_2016.reshape(len(Y_2016),1), X_2016,bws_2016, constant=True).fit()

59.0


In [99]:
out_df_2016 = pd.DataFrame()
for i in results.params:
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare'],i))
    out_df_2016 =  out_df_2016.append(_answer,ignore_index=True)
    
out_df_2016 = out_df_2016.set_index(x_df_2016.index)
out_df_2016.to_excel('coef_2016.xlsx')

tval_df_2016 = pd.DataFrame()
for i in results.tvalues:
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare'],i))
    tval_df_2016 =  tval_df_2016.append(_answer,ignore_index=True)
    
tval_df_2016 = tval_df_2016.set_index(x_df_2016.index)
tval_df_2016.to_excel('tval_2016.xlsx')